# Clang [AArch64](https://en.wikipedia.org/wiki/AArch64)

My personal notes on Clang AArch64

- [Clang](https://clang.llvm.org/) release 15.0.7
- JupyterLab 3.5.2
- [Termux 0.118.0](https://termux.dev/en/)
- Moto G4 with [Android 9.0_r44 ARM64 AOSiP](https://forum.xda-developers.com/t/rom-9-0_r44-arm64-android-open-source-illusion-project-athene-unofficial.3889942/)
- Processor: Snapdragon 617 octa-core ARMv8 Cortex-A53 64-bit
    - Two quad-core clusters 1.2/1.5 GHz

Some references:

- https://stackoverflow.com/questions/200292/process-for-reducing-the-size-of-an-executable
- http://www.muppetlabs.com/~breadbox/software/tiny/
- https://stac47.github.io/c/relocation/elf/tutorial/2018/03/01/understanding-relocation-elf.html
- http://timelessname.com/elfbin/
- https://interrupt.memfault.com/blog/dealing-with-large-symbol-files
- https://www.reddit.com/r/C_Programming/comments/wdag9l/how_to_absolutely_minimize_the_executable/
- http://cs107e.github.io/guides/gcc/

2023-01-28

In [2]:
!inxi

CPU: 2x 4-core AArch64 (-MCP AMP-) speed/min/max: 435/499:403/1651:1210 MHz
Kernel: 3.10.108-lk.r17_rev aarch64 Up: 6d 19h 15m Mem: 1128.3/1843.6 MiB
(61.2%) Storage: 14.56 GiB (186.5% used) Procs: 8 Shell: python3.11
inxi: 3.3.24


In [3]:
!inxi -C

CPU:
  Info: 2x 4-core model: AArch64 bits: 64 type: MCP AMP
  Speed (MHz): avg: 703 min/max: 499:403/1651:1210 cores: 1: 499 2: 499
    3: 806 4: 806 5: 806 6: 806


In [4]:
!lscpu

Architecture:            aarch64
  CPU op-mode(s):        32-bit, 64-bit
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   1,2,4-7
  Off-line CPU(s) list:  0,3
Vendor ID:               ARM
  Model name:            Cortex-A53
    Model:               4
    Thread(s) per core:  1
    Core(s) per cluster: 3
    Socket(s):           -
    Cluster(s):          2
    Stepping:            r0p4
    CPU(s) scaling MHz:  61%
    CPU max MHz:         1651.2000
    CPU min MHz:         0.0000
    Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32


In [1]:
!clang --version

clang version 15.0.7
Target: aarch64-unknown-linux-android24
Thread model: posix
InstalledDir: /data/data/com.termux/files/usr/bin


## tiny.c

In [234]:
%%writefile tiny1.c
int main(void) {
    return 42; 
}

Overwriting tiny.c


In [236]:
!clang -Oz -s tiny1.c

In [232]:
!./a.out ; echo $?

/data/data/com.termux/files/usr/bin/bash: line 1: ./a.out: No such file or directory
127


In [87]:
!wc -c a.out

4248 a.out


In [88]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 4.2K Jan 27 19:35 a.out


In [89]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
   1055	    592	      8	   1655	    677	a.out


In [90]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  00000000000002a8  00000000000002a8  000002a8  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .note.android.ident 00000098  00000000000002c0  00000000000002c0  000002c0  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .dynsym       00000060  0000000000000358  0000000000000358  00000358  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .gnu.version  00000008  00000000000003b8  00000000000003b8  000003b8  2**1
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .gnu.version_r 00000020  00000000000003c0  00000000000003c0  000003c0  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  5 .gnu.hash     0000001c  00000000000003e0  00000000000003e0  000003e0  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  6 .dynstr       00000066  00000000000003fc  000

In [349]:
%%writefile tiny.c
extern void _exit(int);

void _start(void) {
    _exit(42); 
}

Overwriting tiny.c


In [350]:
%%writefile tiny.asm
    .global _start
    .text
_start:
    /* syscall exit (int status) */
    mov    w8, #93  /* exit is syscall #1 */
    mov    x0, #42  /* status  := 42 */
    svc    #0

Overwriting tiny.asm


In [351]:
!as tiny.asm -o tiny.o

\
Linking with clang:

In [352]:
!clang -Wall -s -nostartfiles -nostdlib tiny.o

In [353]:
!./a.out ; echo $?

42


In [354]:
!wc -c a.out

1664 a.out


In [355]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
    122	    160	      0	    282	    11a	a.out


In [356]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 1.7K Jan 27 21:47 a.out


In [357]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  0000000000000238  0000000000000238  00000238  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .dynsym       00000018  0000000000000250  0000000000000250  00000250  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .gnu.hash     0000001c  0000000000000268  0000000000000268  00000268  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .dynstr       00000025  0000000000000284  0000000000000284  00000284  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .text         0000000c  00000000000012ac  00000000000012ac  000002ac  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  5 .dynamic      000000a0  00000000000022b8  00000000000022b8  000002b8  2**3
                  CONTENTS, ALLOC, LOAD, DATA
  6 .data         00000000  0000000000003358  0000000000003358  00

In [358]:
!strip --strip-all --verbose a.out

copy from `a.out' [elf64-littleaarch64] to `stO4z6q4' [elf64-littleaarch64]


In [359]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  0000000000000238  0000000000000238  00000238  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .dynsym       00000018  0000000000000250  0000000000000250  00000250  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .gnu.hash     0000001c  0000000000000268  0000000000000268  00000268  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .dynstr       00000025  0000000000000284  0000000000000284  00000284  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .text         0000000c  00000000000012ac  00000000000012ac  000002ac  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  5 .dynamic      000000a0  00000000000022b8  00000000000022b8  000002b8  2**3
                  CONTENTS, ALLOC, LOAD, DATA
  6 .data         00000000  0000000000003358  0000000000003358  00

\
Linking and gereneting executable direct from assembly and linker

In [360]:
!rm a.out

In [361]:
!ls -lh tiny.o

-rw------- 1 u0_a113 u0_a113 728 Jan 27 21:46 tiny.o


In [362]:
!ld --strip-all --gc-sections -static tiny.o

In [363]:
!./a.out ; echo $?

42


In [364]:
!wc -c a.out

344 a.out


In [365]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         0000000c  0000000000400078  0000000000400078  00000078  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE


In [366]:
!as -al tiny.asm

AARCH64 GAS  tiny.asm 			page 1


   1              	    .global _start
   2              	    .text
   3              	_start:
   4              	    /* syscall exit (int status) */
   5 0000 A80B8052 	    mov    w8, #93  /* exit is syscall #1 */
   6 0004 400580D2 	    mov    x0, #42  /* status  := 42 */
   7 0008 010000D4 	    svc    #0


In [367]:
!objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000000000 <_start>:
   0:	52800ba8 	mov	w8, #0x5d                  	// #93
   4:	d2800540 	mov	x0, #0x2a                  	// #42
   8:	d4000001 	svc	#0x0


In [368]:
!hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  01 00 b7 00 01 00 00 00  00 00 00 00 00 00 00 00  |................|
00000020  00 00 00 00 00 00 00 00  18 01 00 00 00 00 00 00  |................|
00000030  00 00 00 00 40 00 00 00  00 00 40 00 07 00 06 00  |....@.....@.....|
00000040  a8 0b 80 52 40 05 80 d2  01 00 00 d4 00 00 00 00  |...R@...........|
00000050  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000060  00 00 00 00 00 00 00 00  00 00 00 00 03 00 01 00  |................|
00000070  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000080  00 00 00 00 03 00 02 00  00 00 00 00 00 00 00 00  |................|
00000090  00 00 00 00 00 00 00 00  00 00 00 00 03 00 03 00  |................|
000000a0  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
000000b0  01 00 00 00 00 00 01 00  00 00 00 00 00 00 00 00  |................|
000000c0  00 00 00 00 00 00 00 00  04 00 00 00 10 00

In [369]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
     12	      0	      0	     12	      c	a.out


In [370]:
!size -Ax a.out

a.out  :
section   size   addr
.text    0xc    0x0
.data    0x0    0x0
.bss     0x0    0x0
Total    0xc




# C with more aggressive options

In [371]:
%%writefile tiny2.c
extern void _exit(int);

void _start(void) {
    _exit(42); 
}

Writing tiny2.c


In [452]:
%%writefile tiny2.asm
    .global _exit
    .text
_exit:
    /* syscall exit (int status) */
    mov    w8, #93  /* exit is syscall #1 */
    mov    x0, #42  /* status  := 42 */
    svc    #0

Overwriting tiny2.asm


In [453]:
!as tiny2.asm -o tiny2.o

- -ffreestanding is used in embedded systems where there is no operating system, and everything has to be self-contained.
    - void main(void) .
    - The standard header files <stdio.h>, <string.h> and so on, are not to be used.
- On ARM64 the "-Oz -ffreestanding" generated assembly does not include SP stack initialization.
- -fident and -fno-ident control whether the output file contains the compiler name and version information.
- -nostdlib implies the individual options -nodefaultlibs and -nostartfiles .
    - the only libraries linked are exactly those that you explicitly name to the linker using the -l flag .
    - http://cs107e.github.io/guides/gcc/

---

In [450]:
%%bash
clang -Wall -g -Oz -s -static -nostartfiles -nostdlib -ffreestanding \
-fno-ident -fno-asynchronous-unwind-tables  \
-ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all \
tiny2.o tiny2.c

In [451]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 616 Jan 27 22:29 a.out


In [454]:
!./a.out ; echo $?

42


In [455]:
!wc -c a.out

616 a.out


In [456]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  0000000000201120  0000000000201120  00000120  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  1 .comment      00000013  0000000000000000  0000000000000000  00000134  2**0
                  CONTENTS, READONLY


In [457]:
!strip --strip-all --verbose -R .comment a.out

copy from `a.out' [elf64-littleaarch64] to `st227nYp' [elf64-littleaarch64]


In [458]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 520 Jan 27 22:30 a.out


In [459]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
     20	      0	      0	     20	     14	a.out


In [460]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  0000000000201120  0000000000201120  00000120  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE


In [461]:
!objdump -Dz a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000201120 <.text>:
  201120:	52800ba8 	mov	w8, #0x5d                  	// #93
  201124:	d2800540 	mov	x0, #0x2a                  	// #42
  201128:	d4000001 	svc	#0x0
  20112c:	52800540 	mov	w0, #0x2a                  	// #42
  201130:	17fffffc 	b	0x201120


In [462]:
!objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000201120 <.text>:
  201120:	52800ba8 	mov	w8, #0x5d                  	// #93
  201124:	d2800540 	mov	x0, #0x2a                  	// #42
  201128:	d4000001 	svc	#0x0
  20112c:	52800540 	mov	w0, #0x2a                  	// #42
  201130:	17fffffc 	b	0x201120


In [463]:
!hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  2c 11 20 00 00 00 00 00  |........,. .....|
00000020  40 00 00 00 00 00 00 00  48 01 00 00 00 00 00 00  |@.......H.......|
00000030  00 00 00 00 40 00 38 00  04 00 40 00 03 00 02 00  |....@.8...@.....|
00000040  06 00 00 00 04 00 00 00  40 00 00 00 00 00 00 00  |........@.......|
00000050  40 00 20 00 00 00 00 00  40 00 20 00 00 00 00 00  |@. .....@. .....|
00000060  e0 00 00 00 00 00 00 00  e0 00 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  01 00 00 00 04 00 00 00  |................|
00000080  00 00 00 00 00 00 00 00  00 00 20 00 00 00 00 00  |.......... .....|
00000090  00 00 20 00 00 00 00 00  20 01 00 00 00 00 00 00  |.. ..... .......|
000000a0  20 01 00 00 00 00 00 00  00 10 00 00 00 00 00 00  | ...............|
000000b0  01 00 00 00 05 00 00 00  20 01 00 00 00 00 00 00  |........ .......|
000000c0  20 11 20 00 00 00 00 00  20 11 20 00 00 00

---

- -Wl,–gc-sections: Tell the linker to garbage collect and discard unused sections
- -Wl,-static: Link against static libraries. Required for dead-code elimination

In [469]:
%%bash
clang tiny2.o tiny2.c \
-Oz -s -ffreestanding -nostartfiles -nostdlib -static -Wl,--gc-sections

In [470]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 632 Jan 27 22:31 a.out


In [471]:
!./a.out ; echo $?

42


In [472]:
!size a.out

   text	   data	    bss	    dec	    hex	filename
     20	      0	      0	     20	     14	a.out


In [473]:
!objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000201120 <.text>:
  201120:	52800ba8 	mov	w8, #0x5d                  	// #93
  201124:	d2800540 	mov	x0, #0x2a                  	// #42
  201128:	d4000001 	svc	#0x0
  20112c:	52800540 	mov	w0, #0x2a                  	// #42
  201130:	17fffffc 	b	0x201120


In [474]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  0000000000201120  0000000000201120  00000120  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  1 .comment      00000029  0000000000000000  0000000000000000  00000134  2**0
                  CONTENTS, READONLY


In [475]:
!objdump -s a.out


a.out:     file format elf64-littleaarch64

Contents of section .text:
 201120 a80b8052 400580d2 010000d4 40058052  ...R@.......@..R
 201130 fcffff17                             ....            
Contents of section .comment:
 0000 636c616e 67207665 7273696f 6e203135  clang version 15
 0010 2e302e37 00004c69 6e6b6572 3a204c4c  .0.7..Linker: LL
 0020 44203135 2e302e37 00                 D 15.0.7.       


In [476]:
!strip --verbose -R .comment a.out

copy from `a.out' [elf64-littleaarch64] to `stoSzXe0' [elf64-littleaarch64]


In [477]:
!strip --strip-all --verbose -R .comment a.out

copy from `a.out' [elf64-littleaarch64] to `stzu3HC7' [elf64-littleaarch64]


In [478]:
!./a.out ; echo $?

42


In [479]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 520 Jan 27 22:32 a.out


In [480]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  0000000000201120  0000000000201120  00000120  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE


In [481]:
!objdump -s a.out


a.out:     file format elf64-littleaarch64

Contents of section .text:
 201120 a80b8052 400580d2 010000d4 40058052  ...R@.......@..R
 201130 fcffff17                             ....            


In [482]:
!hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  2c 11 20 00 00 00 00 00  |........,. .....|
00000020  40 00 00 00 00 00 00 00  48 01 00 00 00 00 00 00  |@.......H.......|
00000030  00 00 00 00 40 00 38 00  04 00 40 00 03 00 02 00  |....@.8...@.....|
00000040  06 00 00 00 04 00 00 00  40 00 00 00 00 00 00 00  |........@.......|
00000050  40 00 20 00 00 00 00 00  40 00 20 00 00 00 00 00  |@. .....@. .....|
00000060  e0 00 00 00 00 00 00 00  e0 00 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  01 00 00 00 04 00 00 00  |................|
00000080  00 00 00 00 00 00 00 00  00 00 20 00 00 00 00 00  |.......... .....|
00000090  00 00 20 00 00 00 00 00  20 01 00 00 00 00 00 00  |.. ..... .......|
000000a0  20 01 00 00 00 00 00 00  00 10 00 00 00 00 00 00  | ...............|
000000b0  01 00 00 00 05 00 00 00  20 01 00 00 00 00 00 00  |........ .......|
000000c0  20 11 20 00 00 00 00 00  20 11 20 00 00 00

---

In [488]:
%%writefile tiny3.c
#include <stdio.h>

int main(void) {
    puts("Hello World!\n");
}

Overwriting tiny3.c


In [489]:
!clang -Wall tiny3.c

In [492]:
!./a.out

Hello World!



In [494]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 5.9K Jan 27 22:37 a.out


---

In [500]:
%%writefile tiny4.c
#include <stdio.h>

void _start(void) {
    puts("Hello World!\n");
}

Overwriting tiny4.c


In [524]:
%%bash
clang tiny4.c \
-Wall -Oz -s -nostartfiles \
-fno-ident -fno-asynchronous-unwind-tables  \
-ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all \

In [525]:
!strip --strip-all --verbose -R .comment a.out

copy from `a.out' [elf64-littleaarch64] to `stBvxzAr' [elf64-littleaarch64]


In [526]:
!./a.out

Hello World!



In [527]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 2.2K Jan 27 22:53 a.out


In [513]:
!objdump -Dz a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .interp:

0000000000000200 <.interp>:
 200:	7379732f 	.inst	0x7379732f ; undefined
 204:	2f6d6574 	.inst	0x2f6d6574 ; undefined
 208:	2f6e6962 	umlsl	v2.4s, v11.4h, v14.h[6]
 20c:	6b6e696c 	.inst	0x6b6e696c ; undefined
 210:	34367265 	cbz	w5, 6d05c <puts@plt+0x6bd2c>
 214:	Address 0x214 is out of bounds.


Disassembly of section .dynsym:

0000000000000218 <.dynsym>:
 218:	00000000 	udf	#0
 21c:	00000000 	udf	#0
 220:	00000000 	udf	#0
 224:	00000000 	udf	#0
 228:	00000000 	udf	#0
 22c:	00000000 	udf	#0
 230:	00000001 	udf	#1
 234:	00000012 	udf	#18
 238:	00000000 	udf	#0
 23c:	00000000 	udf	#0
 240:	00000000 	udf	#0
 244:	00000000 	udf	#0

Disassembly of section .gnu.version:

0000000000000248 <.gnu.version>:
 248:	00020000 	.inst	0x00020000 ; undefined

Disassembly of section .gnu.version_r:

000000000000024c <.gnu.version_r>:
 24c:	00010001 	.inst	0x00010001 ; undefined
 250:	00000006 	udf	#6
 254:	00000010 	udf	#16


---

In [551]:
%%writefile tiny5.c
#include <stdio.h>

void _start(void) {
    char *s="Hello World!\n";
    write(1,s,strlen(s));
}

Overwriting tiny5.c


In [552]:
%%bash
clang tiny5.c \
-Oz -s -nostartfiles \
-fno-ident -fno-asynchronous-unwind-tables  \
-ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all

tiny5.c:5:5: warning: call to undeclared function 'write'; ISO C99 and later do not support implicit function declarations [-Wimplicit-function-declaration]
    write(1,s,strlen(s));
    ^
1 warning generated.


In [533]:
!strip --strip-all --verbose -R .comment a.out

copy from `a.out' [elf64-littleaarch64] to `stdVAkuP' [elf64-littleaarch64]


In [534]:
!./a.out

Hello World!


In [535]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 2.2K Jan 27 22:59 a.out


In [538]:
%%writefile hello.s
    .data

/* Data segment: define our message string and calculate its length. */
helloworld:
    .ascii        "Hello, World!\n"
helloworld_len = . - helloworld

    .text

/* Our application's entry point. */
.globl _start
_start:
    /* syscall write(int fd, const void *buf, size_t count) */
    mov     x0, #1              /* fd := STDOUT_FILENO */
    ldr     x1, =helloworld     /* buf := msg */
    ldr     x2, =helloworld_len /* count := len */
    mov     w8, #64             /* write is syscall #64 */
    svc     #0                  /* invoke syscall */

    /* syscall exit(int status) */
    mov     x0, #0               /* status := 0 */
    mov     w8, #93              /* exit is syscall #1 */
    svc     #0                   /* invoke syscall */

Writing hello.s


In [540]:
!as hello.s -o hello.o

In [541]:
!ld --strip-all --gc-sections -static hello.o

In [542]:
!./a.out

Hello, World!


In [543]:
!ls -lh a.out

-rwx------ 1 u0_a113 u0_a113 520 Jan 27 23:04 a.out


In [544]:
!objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000030  00000000004000b0  00000000004000b0  000000b0  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  1 .data         0000000e  00000000004100e0  00000000004100e0  000000e0  2**0
                  CONTENTS, ALLOC, LOAD, DATA


In [547]:
!objdump -s a.out


a.out:     file format elf64-littleaarch64

Contents of section .text:
 4000b0 200080d2 e1000058 02010058 08088052   ......X...X...R
 4000c0 010000d4 000080d2 a80b8052 010000d4  ...........R....
 4000d0 e0004100 00000000 0e000000 00000000  ..A.............
Contents of section .data:
 4100e0 48656c6c 6f2c2057 6f726c64 210a      Hello, World!.  


In [549]:
!objdump -D -j .text a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

00000000004000b0 <.text>:
  4000b0:	d2800020 	mov	x0, #0x1                   	// #1
  4000b4:	580000e1 	ldr	x1, 0x4000d0
  4000b8:	58000102 	ldr	x2, 0x4000d8
  4000bc:	52800808 	mov	w8, #0x40                  	// #64
  4000c0:	d4000001 	svc	#0x0
  4000c4:	d2800000 	mov	x0, #0x0                   	// #0
  4000c8:	52800ba8 	mov	w8, #0x5d                  	// #93
  4000cc:	d4000001 	svc	#0x0
  4000d0:	004100e0 	.inst	0x004100e0 ; undefined
  4000d4:	00000000 	udf	#0
  4000d8:	0000000e 	udf	#14
  4000dc:	00000000 	udf	#0


In [550]:
!hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  b0 00 40 00 00 00 00 00  |..........@.....|
00000020  40 00 00 00 00 00 00 00  08 01 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  02 00 40 00 04 00 03 00  |....@.8...@.....|
00000040  01 00 00 00 05 00 00 00  00 00 00 00 00 00 00 00  |................|
00000050  00 00 40 00 00 00 00 00  00 00 40 00 00 00 00 00  |..@.......@.....|
00000060  e0 00 00 00 00 00 00 00  e0 00 00 00 00 00 00 00  |................|
00000070  00 00 01 00 00 00 00 00  01 00 00 00 06 00 00 00  |................|
00000080  e0 00 00 00 00 00 00 00  e0 00 41 00 00 00 00 00  |..........A.....|
00000090  e0 00 41 00 00 00 00 00  0e 00 00 00 00 00 00 00  |..A.............|
000000a0  0e 00 00 00 00 00 00 00  00 00 01 00 00 00 00 00  |................|
000000b0  20 00 80 d2 e1 00 00 58  02 01 00 58 08 08 80 52  | ......X...X...R|
000000c0  01 00 00 d4 00 00 80 d2  a8 0b 80 52 01 00